In [10]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr

목차

- 1. 데이터 처리 (불러오기,추가,수정,변환)
- 2. 플레이리스트로 추천 (협업필터링)

# 데이터 처리

## train,song,genre data 불러오기

In [11]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

## train_df 전처리

In [12]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


### train_df 에 포함된 태그들 (중복포함,중복제거)

In [13]:
# 플레이리스트 곡수 컬럼 추가
train_df['tags_cnt'] = train_df['tags'].map(lambda x : len(x))

# 플레이리스트 태그수 컬럼 추가
train_df['songs_cnt'] = train_df['songs'].map(lambda x : len(x))

### train_df 에 포함된 곡들 (중복포함,중복제거)

In [14]:
from itertools import chain

# 플레이리스트 포함된 노래 중복포함
songs_duplicate = chain.from_iterable(train_df['songs'].tolist())

# 플레이리스트 포함된 노래 중복제거
songs_unique = list(set(songs_duplicate))

### train_df 에 포함된 태그들 (중복포함,중복제거)

In [15]:
# 플레이리스트 포함된 태그 중복포함
tags_duplicate = list(chain.from_iterable(train_df['tags'].tolist()))

# 플레이리스트 포함된 태그 중복제거
tags_unique = list(set(tags_duplicate))

### tag에 새로운 id부여, new_tags_id 컬럼 생성

In [16]:
# { 태그 : 새로운id } 딕셔너리
tag_to_id = dict(zip(tags_unique,range(0,len(tags_unique))))

# { 새로운id : 태그 } 딕셔너리
id_to_tag = dict(zip(range(0,len(tags_unique)),tags_unique))

train_df['new_tags_id'] = train_df['tags'].map(lambda x : [tag_to_id[v] for v in x])

### songs에 새로운 id부여, new_songs_id 컬럼 생성

In [17]:
# { 노래 : 새로운id } 딕셔너리
song_to_id = dict(zip(songs_unique,range(0,len(songs_unique))))

# { 새로운id : 태그 } 딕셔너리
id_to_song = dict(zip(range(0,len(songs_unique)),songs_unique))

train_df['new_songs_id'] = train_df['songs'].map(lambda x : [song_to_id[v] for v in x])

In [18]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,new_tags_id,new_songs_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[8539],"[456704, 112732, 333158, 488440, 258853, 12127..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,2,42,"[15322, 24598]","[375894, 587314, 431997, 104605, 338568, 21226..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,2,28,"[27567, 2605]","[72132, 240434, 144495, 161861, 307991, 222934..."


In [19]:
train_df = train_df[['id','plylst_title','tags','new_tags_id','songs','new_songs_id','tags_cnt','songs_cnt','like_cnt','updt_date']]
train_df.head(3)

,id,plylst_title,tags,new_tags_id,songs,new_songs_id,tags_cnt,songs_cnt,like_cnt,updt_date
0,61281,여행같은 음악,[락],[8539],"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[추억, 회상]","[15322, 24598]","[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[27567, 2605]","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [20]:
train_df.columns = ['플리_id','플리제목','태그','새태그_id','노래_id','새노래_id','태그수','노래수','좋아요수','갱신일']
train_df.head(3)

,플리_id,플리제목,태그,새태그_id,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,61281,여행같은 음악,[락],[8539],"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[추억, 회상]","[15322, 24598]","[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[27567, 2605]","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


### 변수 정리

<현재>

- tags_duplicate : 태그 중복 포함 리스트
- tags_unique : 태그 중복 제거 리스트
- songs_duplicate : 노래 중복 포함 리스트
- songs_unique : 노래 중복 제거 리스트
 
- tag_to_id : { 태그 : 새로운id } 딕셔너리
- id_to_tag : { 새로운id : 태그 } 딕셔너리
- song_to_id : { 노래 : 새로운id } 딕셔너리
- id_to_song : { 새로운id : 노래 } 딕셔너리

# 태그로 노래를 추천해보자

## 태그 출몰횟수 구하기

In [21]:
tag_id_duplicate = list(chain.from_iterable(train_df['새태그_id'].tolist()))
tag_id_unique = list(id_to_tag.keys())

In [22]:
# 각 태그가 몇번 출몰 했는지 세어 놓은 딕셔너리

from collections import Counter

tag_count = dict(Counter(tag_id_duplicate))

tag_count = {i:tag_count[i] for i in range(len(tag_count))}

In [25]:
# 태그_id에 달린 노래id들을 넣을 딕셔너리 초기화
tag_id_to_song_ids_dict = {i:[] for i in range(len(tag_id_unique))}

In [26]:
# 태그_id에 달린 장르id들을 list로 넣어주기
for tag_ids , song_ids in zip(train_df['새태그_id'].tolist(),train_df['새노래_id'].tolist()):
    for tag_id in tag_ids:
        tag_id_to_song_ids_dict[tag_id].extend(song_ids)

In [29]:
for i in range(len(tag_id_to_song_ids_dict)):
    song_count = Counter(tag_id_to_song_ids_dict[i])
    song_count_sorted = sorted(song_count.items(),key=(lambda x:x[1]),reverse=True)

    temp_list = []
    
    if len(song_count_sorted) > 9:
        for k,v in song_count_sorted[:10]:
            temp_list.append((k,v))
    else:
        for k,v in song_count_sorted:
            temp_list.append((k,v))
            
    tag_id_to_song_ids_dict[i] = temp_list

In [30]:
# 포함된 노래의 개수를 가지고 있는 list
song_len = [len(v) for v in  tag_id_to_song_ids_dict.values()]

In [31]:
#  태그별 장르포함 횟수만큼 반복  * 2번 태그는 7번 나왔으니 2를 7번 찍기
row = np.repeat(range(len( tag_id_to_song_ids_dict)),song_len)
len(row)

290304

In [32]:
col =  [song[0] for k,v in tag_id_to_song_ids_dict.items() for song in v]
len(col)

290304

In [33]:
# 태그 마다 달린 노래의 총길이 만큼 1찍기
dat = np.repeat(1, sum(song_len))
len(dat)

290304

In [34]:
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(train_df['태그수'].sum(), len(song_to_id)))

train_tags_A_T = train_tags_A.T.tocsr()

In [35]:
def recom_genres(tag):


    my_song_ids = tag_id_to_song_ids_dict[tag_to_id[tag]]
    
    # 내가 뽑은 장르 id 만 1인 1차원 희소행렬

    p = np.zeros((len(song_to_id),1))

    for id,count in my_song_ids:
        p[id]=1
        
    # 희소행렬과 dot 연산을 통해 각 태그와 겹치는 장르들의 갯수를 추출
    val = train_tags_A.dot(p).reshape(-1)
    
    cand_gen = train_tags_A_T.dot(val)
    
    # 1차원으로 바꿔서 중복된 장르를 가진 태그 상위 150개 도출
    cand_gen_idx = cand_gen.reshape(-1).argsort()[-150:][::-1]

    # 비슷한 장르 10개 도출
    cand_gen_idx = cand_gen_idx[:10]

    cand_gen_idx
    
    # 원래 노래 id값으로 복원
    result_ids = [id_to_song[i] for i in cand_gen_idx]
    
    
    #result_songs = [(id_to_song[i]) for i in result_ids]
    
    return result_ids

In [36]:
song_ids = recom_genres('백예린')

In [37]:
song_df.loc[song_ids]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
27469,"[GN0401, GN0403]",20160620,Bye bye my blue,2692170,[698776],Bye bye my blue,[GN0400],[백예린],27469
446812,"[GN0401, GN0403]",20151130,FRANK,2653573,[698776],우주를 건너,[GN0400],[백예린],446812
342835,"[GN0401, GN0403]",20160620,Bye bye my blue,2692170,[698776],그의 바다,[GN0400],[백예린],342835
300553,"[GN0401, GN0402]",20190318,Our love is great,10262378,[698776],그건 아마 우리의 잘못은 아닐 거야,[GN0400],[백예린],300553
144663,"[GN0105, GN0101]",20170324,밤편지,10047890,[261143],밤편지,[GN0100],[아이유],144663
285104,"[GN0105, GN0101]",20160620,Bye bye my blue,2692170,[698776],Zero,[GN0100],[백예린],285104
357367,"[GN0401, GN0403]",20160621,비,2692501,[752425],비,[GN0400],[폴킴],357367
505036,"[GN0401, GN0403, GN0402]",20160324,130 mood : TRBL,2674623,[880630],D (half moon) (Feat. 개코),[GN0400],[DEAN],505036
141459,"[GN0105, GN0101]",20161207,Love you on Christmas,10021255,[698776],November song,[GN0100],[백예린],141459
419602,"[GN0401, GN0402]",20191210,Every letter I sent you.,10362776,[698776],Square (2017),[GN0400],[백예린],419602
